In [1]:
'''
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install tensorflow
!pip install keras
!pip install matplotlib
!pip install seaborn
!pip install plotly
!pip install tqdm
'''

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import json
from collections import defaultdict

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential

from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
tf.__version__

'2.9.1'

In [3]:
print(tf.config.list_physical_devices('GPU'))

[]


In [6]:
# get [log key, delta time] as input for deeplog
input_dir  = '../Datasets/hdfs/'
output_dir = '../output/hdfs/'  # The output directory of parsing results
log_file   = "HDFS.log"  # The input log file name

log_structured_file = output_dir + log_file + "_structured.csv"
log_labeled_file = output_dir + log_file + "_labeled.csv"
log_sequence_file = output_dir + "hdfs_sequence.csv"

### Configuring TPU's as we will be using Bert 

In [4]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [12]:

#train = logFile2DataFrame('../Datasets/HDFS/HDFS.log')
#validation = pd.read_csv('../Datasets/Sentiment/validation.csv')
#test = pd.read_csv('../Datasets/Sentiment/test.csv')

In [14]:
import re
import os

def load_data(log_format):
    headers, regex = generate_logformat_regex(log_format)
    df_log = log_to_dataframe(os.path.join(input_dir, log_file), regex, headers, log_format)
    df_log.to_csv(log_structured_file, index=None)
    return df_log

def log_to_dataframe( log_file, regex, headers, logformat):
    """ Function to transform log file to dataframe
    """
    log_messages = []
    linecount = 0
    cnt = 0
    with open(log_file, 'r') as fin:
        for line in fin.readlines():
            cnt += 1
            try:
                match = regex.search(line.strip())
                message = [match.group(header) for header in headers]
                log_messages.append(message)
                linecount += 1
            except Exception as e:
                # print("\n", line)
                # print(e)
                pass
    print("Total size after encoding is", linecount, cnt)
    logdf = pd.DataFrame(log_messages, columns=headers)
    logdf.insert(0, 'LineId', None)
    logdf['LineId'] = [i + 1 for i in range(linecount)]
    return logdf

def generate_logformat_regex( logformat):
    """ Function to generate regular expression to split log messages
    """
    headers = []
    splitters = re.split(r'(<[^<>]+>)', logformat)
    regex = ''
    for k in range(len(splitters)):
        if k % 2 == 0:
            splitter = re.sub(' +', '\\\s+', splitters[k])
            regex += splitter
        else:
            header = splitters[k].strip('<').strip('>')
            regex += '(?P<%s>.*?)' % header
            headers.append(header)
    regex = re.compile('^' + regex + '$')
    return headers, regex


In [15]:
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
train = load_data(log_format)
train.head()


Total size after encoding is 2270605 2270605


,LineId,Date,Time,Pid,Level,Component,Content
0,1,081109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...
1,2,081109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...
2,3,081109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...
3,4,081109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...
4,5,081109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...


In [17]:

def Merge_Sequences_by_BlockId(log_file, window='session'):
    assert window == 'session', "Only window=session is supported for HDFS dataset."
    print("Loading", log_file)
    df = pd.read_csv(log_file, engine='c',
            na_filter=False, memory_map=True, dtype={'Date':object, "Time": object})

    df['BlockId']= "NULL"
    
    data_dict = defaultdict(list) #preserve insertion order of items
    for idx, row in tqdm(df.iterrows()):
        blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
        blkId_set = set(blkId_list)
        for blk_Id in blkId_set:
            data_dict[blk_Id].append(row["Content"])

    data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])  
    data_df.to_csv(log_sequence_file, index=None)
    return data_df

train= Merge_Sequences_by_BlockId(log_structured_file)
train.head(5)


Loading ../output/hdfs/HDFS.log_structured.csv


2270605it [02:00, 18839.12it/s]


,BlockId,EventSequence
0,blk_-1608999687919862906,[Receiving block blk_-1608999687919862906 src:...
1,blk_7503483334202473044,[Receiving block blk_7503483334202473044 src: ...
2,blk_-3544583377289625738,[Receiving block blk_-3544583377289625738 src:...
3,blk_-9073992586687739851,[Receiving block blk_-9073992586687739851 src:...
4,blk_7854771516489510256,[Receiving block blk_7854771516489510256 src: ...


In [31]:
def Assign_Labels_to_Sequence(hdfs_sequence_file, n=None, ratio=0.3):
    blk_label_dict = {}
    blk_label_file = os.path.join(input_dir, "anomaly_label.csv")
    blk_df = pd.read_csv(blk_label_file)
    for _ , row in tqdm(blk_df.iterrows()):
        blk_label_dict[row["BlockId"]] = 1 if row["Label"] == "Anomaly" else 0

    seq = pd.read_csv(hdfs_sequence_file)
    seq["Label"] = seq["BlockId"].apply(lambda x: blk_label_dict.get(x)) #add label to the sequence of each blockid

    print("Generate sequence and labels done")
    seq.to_csv(log_labeled_file, index=None)
    return seq

train = Assign_Labels_to_Sequence(log_sequence_file)
train.head()

575061it [00:32, 17926.03it/s]


Generate sequence and labels done


,BlockId,EventSequence,Label
0,blk_-1608999687919862906,['Receiving block blk_-1608999687919862906 src...,0.0
1,blk_7503483334202473044,['Receiving block blk_7503483334202473044 src:...,0.0
2,blk_-3544583377289625738,['Receiving block blk_-3544583377289625738 src...,1.0
3,blk_-9073992586687739851,['Receiving block blk_-9073992586687739851 src...,0.0
4,blk_7854771516489510256,['Receiving block blk_7854771516489510256 src:...,0.0


In [32]:
train.isnull().values.any()

True

In [41]:
train['BlockId'].iloc[-1]

'blk_-7718737206603036'

In [33]:
train.loc[train['BlockId'].eq('blk_-7718737206603036')]

,BlockId,EventSequence,Label
109185,blk_-7718737206603036,['Deleting block blk_-7718737206603036'],NaN


#### Remove any NULL Values

In [45]:
train = train.dropna()
train.shape

(109185, 3)

### Check the maximum number of words in a Block

In [46]:
train['EventSequence'].apply(lambda x:len(str(x).split())).max()

1787

In [47]:
#Writing a function for getting auc score for validation

def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [48]:
# Data Preparation

xtrain, xvalid, ytrain, yvalid = train_test_split(train.EventSequence.values, train.Label.values, 
                                                  stratify=train.Label.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

### Simple RNN

In [49]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 10

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [145]:
xvalid_pad

array([[    12,     13,      5, ...,     63,    141,      9],
       [    12,     13,      5, ...,     52,    137,      9],
       [ 45108,    114,    112, ...,    353,      1, 194698],
       ...,
       [    12,     13,      5, ...,     95,    150,      9],
       [    12,     13,      5, ...,     76,     79,      9],
       [    12,     13,      5, ...,     73,    124,      9]])

In [50]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     10,
                     input_length=max_len))
    model.add(SimpleRNN(10))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 10)            2611890   
                                                                 
 simple_rnn (SimpleRNN)      (None, 10)                210       
                                                                 
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 2,612,111
Trainable params: 2,612,111
Non-trainable params: 0
_________________________________________________________________
CPU times: total: 406 ms
Wall time: 560 ms


In [51]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/5
1365/1365 [==============================] - 68s 49ms/step - loss: 0.1317 - accuracy: 0.9700
Epoch 2/5
1365/1365 [==============================] - 63s 46ms/step - loss: 0.0946 - accuracy: 0.9774
Epoch 3/5
1365/1365 [==============================] - 64s 47ms/step - loss: 0.0801 - accuracy: 0.9824
Epoch 4/5
1365/1365 [==============================] - 65s 48ms/step - loss: 0.0758 - accuracy: 0.9843
Epoch 5/5
1365/1365 [==============================] - 71s 52ms/step - loss: 0.0742 - accuracy: 0.9849


In [52]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

683/683 [==============================] - 2s 3ms/step
Auc: 0.78%


In [53]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': roc_auc(scores,yvalid)})

In [54]:
xtrain_seq[:1]

[[7,
  8,
  40,
  42,
  41,
  101,
  43,
  44,
  46,
  100,
  53,
  2720,
  32,
  45,
  2721,
  1,
  200037,
  19,
  4,
  1,
  10163,
  17,
  2,
  3,
  59,
  120,
  76286,
  18,
  2,
  3,
  59,
  120,
  9,
  19,
  4,
  1,
  10163,
  17,
  2,
  3,
  76,
  110,
  115162,
  18,
  2,
  3,
  76,
  110,
  9,
  19,
  4,
  1,
  10163,
  17,
  2,
  3,
  76,
  110,
  133584,
  18,
  2,
  3,
  76,
  110,
  9,
  22,
  32,
  16,
  4,
  1,
  10163,
  23,
  21,
  4,
  1,
  10163,
  11,
  10,
  29,
  20,
  2,
  3,
  59,
  222,
  22,
  38,
  16,
  4,
  1,
  10163,
  23,
  21,
  4,
  1,
  10163,
  11,
  10,
  29,
  20,
  2,
  3,
  76,
  219,
  7,
  8,
  24,
  25,
  26,
  2,
  3,
  59,
  120,
  6,
  12,
  13,
  5,
  1,
  10163,
  10,
  30,
  7,
  8,
  24,
  25,
  26,
  2,
  3,
  76,
  110,
  6,
  12,
  13,
  5,
  1,
  10163,
  10,
  30,
  7,
  8,
  24,
  25,
  26,
  2,
  3,
  50,
  138,
  6,
  12,
  13,
  5,
  1,
  10163,
  10,
  30,
  22,
  47,
  16,
  4,
  1,
  10163,
  23,
  21,
  4,
  1,
  10163,
  1

#### Embeddings

In [57]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('../Datasets/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [07:17, 5022.28it/s]


Found 2196016 word vectors.


### LSTM

In [78]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 261188/261188 [00:00<00:00, 637175.30it/s]


In [80]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(10, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 10, 300)           78356700  
                                                                 
 lstm_1 (LSTM)               (None, 10)                12440     
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 78,369,151
Trainable params: 12,451
Non-trainable params: 78,356,700
_________________________________________________________________
CPU times: total: 4.55 s
Wall time: 1.66 s


In [81]:
model.fit(xtrain_pad, ytrain,epochs=5, batch_size=64*strategy.num_replicas_in_sync)


Epoch 1/5
1365/1365 [==============================] - 24s 15ms/step - loss: 0.1389 - accuracy: 0.9671
Epoch 2/5
1365/1365 [==============================] - 23s 17ms/step - loss: 0.1229 - accuracy: 0.9712
Epoch 3/5
1365/1365 [==============================] - 23s 17ms/step - loss: 0.1219 - accuracy: 0.9713
Epoch 4/5
1365/1365 [==============================] - 23s 17ms/step - loss: 0.1217 - accuracy: 0.9713
Epoch 5/5
1365/1365 [==============================] - 24s 17ms/step - loss: 0.1213 - accuracy: 0.9713


In [82]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

683/683 [==============================] - 3s 4ms/step
Auc: 0.81%


In [83]:
scores_model.append({'Model': 'LSTM','AUC_Score': roc_auc(scores,yvalid)})


### Gated Recurrent Unit - GRU's

In [84]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
     model = Sequential()
     model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
     model.add(SpatialDropout1D(0.3))
     model.add(GRU(10))
     model.add(Dense(1, activation='sigmoid'))

     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 10, 300)           78356700  
                                                                 
 spatial_dropout1d (SpatialD  (None, 10, 300)          0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 10)                9360      
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 78,366,071
Trainable params: 9,371
Non-trainable params: 78,356,700
_________________________________________________________________
CPU times: total: 4.69 s
Wall time: 1.91 s


In [85]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)


Epoch 1/5
1365/1365 [==============================] - 16s 9ms/step - loss: 0.1367 - accuracy: 0.9675
Epoch 2/5
1365/1365 [==============================] - 14s 10ms/step - loss: 0.1215 - accuracy: 0.9713
Epoch 3/5
1365/1365 [==============================] - 12s 9ms/step - loss: 0.1207 - accuracy: 0.9713
Epoch 4/5
1365/1365 [==============================] - 11s 8ms/step - loss: 0.1200 - accuracy: 0.9714
Epoch 5/5
1365/1365 [==============================] - 12s 9ms/step - loss: 0.1199 - accuracy: 0.9715


In [86]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

683/683 [==============================] - 3s 3ms/step
Auc: 0.81%


In [87]:
scores_model.append({'Model': 'GRU','AUC_Score': roc_auc(scores,yvalid)})


In [88]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.7754204216597802},
 {'Model': 'LSTM', 'AUC_Score': 0.8143838216079735},
 {'Model': 'GRU', 'AUC_Score': 0.8145220293622619}]

### Bi-Directional RNN's

In [90]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(10, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 10, 300)           78356700  
                                                                 
 bidirectional (Bidirectiona  (None, 20)               24880     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 78,381,601
Trainable params: 24,901
Non-trainable params: 78,356,700
_________________________________________________________________
CPU times: total: 4.31 s
Wall time: 1.43 s


In [91]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)


Epoch 1/5
1365/1365 [==============================] - 39s 23ms/step - loss: 0.1323 - accuracy: 0.9666
Epoch 2/5
1365/1365 [==============================] - 34s 25ms/step - loss: 0.1218 - accuracy: 0.9713
Epoch 3/5
1365/1365 [==============================] - 44s 32ms/step - loss: 0.1212 - accuracy: 0.9713
Epoch 4/5
1365/1365 [==============================] - 54s 40ms/step - loss: 0.1207 - accuracy: 0.9713
Epoch 5/5
1365/1365 [==============================] - 68s 50ms/step - loss: 0.1204 - accuracy: 0.9713


In [92]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

683/683 [==============================] - 5s 6ms/step
Auc: 0.82%


In [93]:
scores_model.append({'Model': 'Bi-directional LSTM','AUC_Score': roc_auc(scores,yvalid)})


### Seq2Seq Model Architecture

In [ ]:
### code need to be implemented still

In [104]:
!pip install Jinja2
!pip install nbformat
!pip install  "nbformat>=4.2.0"

#Visualization of Results obtained from various Deep learning models
results = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Blues')

You should consider upgrading via the 'C:\Jad\AI_Projects\AnomalyDetection_Transformers\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Jad\AI_Projects\AnomalyDetection_Transformers\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Jad\AI_Projects\AnomalyDetection_Transformers\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


,Model,AUC_Score
3,Bi-directional LSTM,0.817154
2,GRU,0.814522
1,LSTM,0.814384
0,SimpleRNN,0.775420


### BERT
### =================================

In [4]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from transformers import AutoModel, AutoTokenizer

from tokenizers import BertWordPieceTokenizer

In [13]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q tf-models-official==2.7.0

import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

  error: subprocess-exited-with-error
  
  × Building wheel for pycocotools (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\pycocotools
      copying pycocotools\coco.py -> build\lib.win-amd64-cpython-310\pycocotools
      copying pycocotools\cocoeval.py -> build\lib.win-amd64-cpython-310\pycocotools
      copying pycocotools\mask.py -> build\lib.win-amd64-cpython-310\pycocotools
      copying pycocotools\__init__.py -> build\lib.win-amd64-cpython-310\pycocotools
      running build_ext
      skipping 'pycocotools\_mask.c' Cython extension (up-to-date)
      building 'pycocotools._mask' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end

NotFoundError: c:\Jad\AI_Projects\AnomalyDetection_Transformers\.venv\lib\site-packages\tensorflow_text\python\ops\_regex_split_ops.so not found

In [7]:
train = pd.read_csv(log_labeled_file)
train.head()

,BlockId,EventSequence,Label
0,blk_-1608999687919862906,['Receiving block blk_-1608999687919862906 src...,0.0
1,blk_7503483334202473044,['Receiving block blk_7503483334202473044 src:...,0.0
2,blk_-3544583377289625738,['Receiving block blk_-3544583377289625738 src...,1.0
3,blk_-9073992586687739851,['Receiving block blk_-9073992586687739851 src...,0.0
4,blk_7854771516489510256,['Receiving block blk_7854771516489510256 src:...,0.0


In [8]:
from sklearn.model_selection import train_test_split

# set aside 20% of train and test data for evaluation
X_train, X_test, y_train, y_test = train_test_split(train.EventSequence, train.Label,  test_size=0.2, shuffle = True, random_state = 8)

# Use the same function above for the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2


print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))

X_train shape: (65511,)
y_train shape: (65511,)
X_test shape: (21838,)
y_test shape: (21838,)
X_val shape: (21837,)
y_val shape: (21837,)


In [9]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)


In [157]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [158]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync


In [171]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [182]:
MAX_LEN = 10000
###train['EventSequence'].apply(lambda x:len(str(x).split())).max()

# First load the real tokenizer
model_name= 'distilbert-base-multilingual-cased'
model = transformers.DistilBertModel.from_pretrained(model_name)
tokenizer = transformers.DistilBertTokenizer.from_pretrained(model_name)



Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [196]:
type(str(X_train))

str

In [197]:
str(X_train)

"77276     ['BLOCK* NameSystem.allocateBlock: /user/root/...\n52708     ['BLOCK* NameSystem.allocateBlock: /user/root/...\n14146     ['Receiving block blk_-2354440619947518714 src...\n93357     ['BLOCK* NameSystem.allocateBlock: /user/root/...\n32688     ['Receiving block blk_-2166134416691254576 src...\n                                ...                        \n45701     ['BLOCK* NameSystem.allocateBlock: /user/root/...\n21200     ['BLOCK* NameSystem.allocateBlock: /user/root/...\n108727    ['BLOCK* NameSystem.allocateBlock: /user/root/...\n8797      ['BLOCK* NameSystem.allocateBlock: /user/root/...\n81441     ['BLOCK* NameSystem.allocateBlock: /user/root/...\nName: EventSequence, Length: 65511, dtype: object"

In [200]:
x_train = tokenizer(str(X_train), padding=True, truncation=True, return_tensors="tf")
x_valid = tokenizer(str(X_val), padding=True, truncation=True, return_tensors="tf")
#x_train = fast_encode(X_train.astype(str), fast_tokenizer, maxlen=MAX_LEN)
#x_valid = fast_encode(X_val.astype(str), fast_tokenizer, maxlen=MAX_LEN)
#x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = y_train.values
y_valid = y_val.values

In [201]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

ValueError: Dimensions 1 and 65511 are not compatible

In [ ]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)